In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor,LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl

import kaggle_evaluation.mcts_inference_server

In [ ]:
train_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"
test_path="/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv"

In [ ]:
train_data=pd.read_csv(train_path)
test_data =pd.read_csv(test_path)

In [ ]:
train_data 

In [ ]:
train_data.describe()

In [ ]:
print(len(train_data["agent2"].unique()))

In [ ]:
def summarize_dataframe_info(df):
    table=pd.DataFrame(columns=["Column","Data Type","Null count"])
    Dtype=""
    null=0
    for column in df.columns:
        Dtype=df[column].dtype
        null=df[column].isnull().sum()
        new_row = {"Column": column, "Data Type": Dtype,"Null count":null}
        table = pd.concat([table, pd.DataFrame([new_row])], ignore_index=True)
    return table
        
        

In [ ]:
info=summarize_dataframe_info(train_data)

In [ ]:
info[info["Null count"]>=1]

In [ ]:
info[info["Data Type"]=="object"]

In [ ]:
train_data["GameRulesetName"].unique()

In [ ]:
train_data["agent1"].unique()

Drop columns that have 95% or more identical values and columns with all null values.

In [ ]:
def drop_column_name(df):
    drop_column=[]
    for column in df.columns:
        if df[column].value_counts().max()/233234 >= .95 or len(df[column].value_counts())==0:
            drop_column.append(column)
    return drop_column
drop_column=drop_column_name(train_data)
print (drop_column)

add some column we will not use

In [ ]:
drop_column.extend(["Id","agent1","agent2","EnglishRules","LudRules","num_wins_agent1","num_draws_agent1","num_losses_agent1"])

In [ ]:
print (len(drop_column)) 

In [ ]:
train=train_data.drop(columns=drop_column, errors='ignore')


In [ ]:
le = LabelEncoder()

train["GameRulesetName"] = le.fit_transform(train["GameRulesetName"])

In [ ]:
test=test_data.drop(columns=drop_column, errors='ignore')
test["GameRulesetName"] = le.transform(test["GameRulesetName"])

In [ ]:
train

In [ ]:
test

In [ ]:
X = train.drop('utility_agent1', axis=1)  # Features
y = train['utility_agent1'] 

In [ ]:
X_train,X_valid,y_train,y_valid  =train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = CatBoostRegressor(iterations=1000, 
                          learning_rate=0.05, 
                          depth=6)



In [ ]:

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=True)

In [ ]:
def clean_data(test_df):
    """
    Cleans the test dataframe by converting all applicable columns to numeric and handling errors.
    """
    # Convert columns to numeric, coerce invalid values to NaN
    test_df = test_df.apply(pd.to_numeric, errors='coerce')
    
    # Optionally, fill NaN values with 0 or some other placeholder value
    test_df = test_df.fillna(0)
    
    return test_df

In [ ]:
import os
import polars as pl
import kaggle_evaluation.mcts_inference_server




# Define the predict function as required by the MCTSInferenceServer
def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):

    test_df = test.to_pandas()

    # Clean the data
    test_df = clean_data(test_df)

    # Make predictions using the trained model
    predictions = model.predict(test_df)

    # Summation step: add a constant value to the predictions
    predictions = predictions + 0.123

    # Prepare the final submission format
    submission = sample_sub.with_columns(pl.Series("utility_agent1", predictions))
    
    return submission


In [ ]:
# Setup inference server for batch inference
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

# Check the environment and serve/run locally based on competition conditions
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )